In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import sys
sys.path.append("/meshAfterParty/")

In [7]:
import datajoint_utils as du
import datajoint as dj

In [25]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-12 18:20:56,013 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-12 18:20:56,308 - settings - Setting enable_python_native_blobs to True


# Inspecting Baylor Soma

In [87]:
(schema.jobs & "table_name='__baylor_segment_centroid'")#.delete()# & "status='error'"

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__baylor_segment_centroid,3dc5a913a94fb86965027f55cf33b909,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.167,at-node40,29,34555,2021-01-13 00:50:23
__baylor_segment_centroid,e064d77660fde6bce765b1a5726a8709,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.152,at-node25,30,34564,2021-01-13 00:50:54
__baylor_segment_centroid,f8342b6c2947d01ddeeef7241ea2b8a1,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.128,at-node1,29,34574,2021-01-13 00:50:53


In [88]:
minnie.BaylorSegmentCentroid()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),soma_index index given to this soma to account for multiple somas in one base semgnet,ver the version number of the materializaiton,centroid_x (EM voxels),centroid_y (EM voxels),centroid_z (EM voxels),n_vertices number of vertices,n_faces number of faces,mesh datajoint adapter to get the somas mesh objects,multiplicity the number of somas found for this base segment,sdf sdf width value for the soma,volume the volume in billions (10*9 nm^3) of the convex hull,max_side_ratio the maximum of the side length ratios used for check if soma,bbox_volume_ratio ratio of bbox (axis aligned) volume to mesh volume to use for check if soma,max_hole_length euclidean distance of the maximum hole size,run_time the amount of time to run (seconds)


In [70]:
minnie.BaylorSegmentCentroid()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),soma_index index given to this soma to account for multiple somas in one base semgnet,ver the version number of the materializaiton,centroid_x (EM voxels),centroid_y (EM voxels),centroid_z (EM voxels),n_vertices number of vertices,n_faces number of faces,mesh datajoint adapter to get the somas mesh objects,multiplicity the number of somas found for this base segment,sdf sdf width value for the soma,volume the volume in billions (10*9 nm^3) of the convex hull,max_side_ratio the maximum of the side length ratios used for check if soma,bbox_volume_ratio ratio of bbox (axis aligned) volume to mesh volume to use for check if soma,max_hole_length euclidean distance of the maximum hole size,run_time the amount of time to run (seconds)
864691134947393276,0,0.25,1,29.00,151195,204425,23078,19028,37767,=BLOB=,1,0.818,1774.9898067813228,1.138,5.6,147474.906,565.3138
864691135012398582,0,0.25,1,29.00,256397,172021,21233,20248,40125,=BLOB=,2,0.892,1528.2356333825364,1.206,4.128,55918.289,681.7621
864691135012398582,0,0.25,2,29.00,259228,172686,21595,19340,38392,=BLOB=,2,0.86,1381.4758267331094,1.183,3.825,238864.725,681.7621
864691135012471798,0,0.25,1,29.00,148351,160466,23686,19836,39237,=BLOB=,2,0.79,1359.4990314786876,1.319,3.765,1127255.427,592.3391
864691135012471798,0,0.25,2,29.00,146233,158684,23599,12911,25158,=BLOB=,2,0.79,906.6300003418867,1.284,4.949,2089422.601,592.3391
864691135012571126,0,0.25,1,29.00,144640,135833,22253,37841,74765,=BLOB=,1,0.792,2885.202064287169,1.833,6.681,401769.228,751.079
864691135012905110,0,0.25,1,29.00,179874,177040,16494,29103,56533,=BLOB=,1,0.787,2002.283149743836,1.254,4.476,1446223.939,451.3164
864691135065018692,0,0.25,1,29.00,240123,201750,26340,35489,69644,=BLOB=,2,0.736,3814.666967808354,1.85,6.438,407924.598,481.6432
864691135065018692,0,0.25,2,29.00,240477,186644,26172,12756,24151,=BLOB=,2,0.399,1003.8413342951484,1.519,5.534,5567742.11,481.6432
864691135065024068,0,0.25,1,29.00,256704,178828,18168,46789,91851,=BLOB=,1,0.746,4869.565858574333,1.178,5.073,7651221.993,777.9455


In [86]:
schema.external['somas'].delete(delete_external_files=True)

100%|██████████| 180/180 [00:02<00:00, 67.46it/s]


[]

In [85]:
minnie.BaylorSegmentCentroid.delete()

INFO - 2021-01-13 06:50:59,505 - connection - Transaction started


About to delete:
`microns_minnie65_02`.`__baylor_segment_centroid`: 180 items
Proceed? [yes, No]: yes


INFO - 2021-01-13 06:51:00,641 - connection - Transaction committed and closed.


Committed.


In [82]:
minnie.NeuronGliaNuclei.delete()

INFO - 2021-01-13 06:50:36,766 - connection - Transaction started
INFO - 2021-01-13 06:50:36,776 - connection - Transaction cancelled. Rolling back ...


About to delete:
Nothing to delete


In [83]:
schema.external['faces'].delete(delete_external_files=True)

0it [00:00, ?it/s]


[]

# Inspecting a soma extraction

In [45]:
import system_utils as su

In [46]:
glia_pieces= su.decompress_pickle("glia_pieces")
nuclei_pieces= su.decompress_pickle("nuclei_pieces")
total_soma_list = su.decompress_pickle("total_soma_list")

In [47]:
curr_mesh = du.fetch_segment_id_mesh(864691135403794158)

In [48]:
import neuron_visualizations as nviz

In [49]:
nviz.plot_objects(curr_mesh)

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [51]:
nviz.plot_objects(meshes=total_soma_list)

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


# Inspecting Decomposition

In [92]:
#schema.external['decomposition'].delete(delete_external_files=True)

100%|██████████| 80/80 [00:01<00:00, 64.31it/s]


[]

In [95]:
minnie.Decomposition.drop()

`microns_minnie65_02`.`__decomposition` (0 tuples)
`microns_minnie65_02`.`__neuron_split_suggestions` (0 tuples)
Proceed? [yes, No]: yes


INFO - 2021-01-13 08:13:30,062 - table - Dropped table `microns_minnie65_02`.`__neuron_split_suggestions`
INFO - 2021-01-13 08:13:30,111 - table - Dropped table `microns_minnie65_02`.`__decomposition`


Tables dropped.  Restart kernel.


In [94]:
(schema.jobs & "table_name='__decomposition'").delete() #& "status='error'"

# Inspecting Split Suggestions

In [26]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-12 18:21:13,220 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-12 18:21:13,512 - settings - Setting enable_python_native_blobs to True


In [34]:
len(minnie.NeuronSplitSuggestions())

76

In [32]:
(schema.jobs & "table_name='__neuron_split_suggestions'") & "status='error'"

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__neuron_split_suggestions,087eb9c97f37e9b414e0649b41e5cac2,error,=BLOB=,Exception: base_node (40) and upstream_node (19) are not connected in the concept network,=BLOB=,celiib@10.21.18.111,71caefafdeec,26880,31121,2021-01-12 12:23:25
__neuron_split_suggestions,52a081c3587840e9f0981196ec9f1180,error,=BLOB=,Exception: base_node (74) and upstream_node (78) are not connected in the concept network,=BLOB=,celiib@10.21.18.111,71caefafdeec,26340,31116,2021-01-12 12:50:17
